# Klassifikation der Anfänge von Kurzgeschichten nach ihrem Informationsgrad

In [1]:
pip install openai


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

# Loading Data

In [4]:
file_path = 'Kurzgeschichten_Anfaenge_DHD_2025.xlsx'

In [5]:
df = pd.read_excel(file_path)

In [6]:
df.head()

,Autor,Titel,Absatz,Eingangssatz,Person,Gegenstand,Ort,Zeit,Score,Klassifikation
0,Böll,Der Dieb,"Am meisten war ich darüber erstaunt, wie schne...","Am meisten war ich darüber erstaunt, wie schne...",1,1,0,0,2,sehr wenig Information
1,Böll,Aus der »Vorzeit«,Nachdem die Kompanie vom Kasernenhof abgerückt...,Nachdem die Kompanie vom Kasernenhof abgerückt...,1,2,2,0,5,wenig Information
2,Böll,Im Käfig,Irgendeiner stand am Zaun und blickte grübelnd...,Irgendeiner stand am Zaun und blickte grübelnd...,1,2,2,0,5,wenig Information
3,Böll,In guter Hut,"Als ich wach wurde, wollte ich es zuerst gar n...","Als ich wach wurde, wollte ich es zuerst gar n...",1,2,0,0,3,sehr wenig Information
4,Böll,Todesursache: Hakennase,Als der Leutnant Hegemüller in sein Quartier z...,Als der Leutnant Hegemüller in sein Quartier z...,2,2,2,0,6,wenig Information


In [7]:
df.Klassifikation.value_counts()

Klassifikation
sehr wenig Information    25
wenig Information         22
viel Information           8
Name: count, dtype: int64

## gpt-4 SetUp

The following lines of code and prompts are based on the hints and instructions from the aformentioned course: https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/2/guidelines

In [8]:
import os
import openai

openai.api_key = os.getenv('MY_OPENAI')


In [9]:
def get_completion(prompt, model="gpt-4o"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

# Evaluating

In [10]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

In [11]:
results = []
for text in df.Eingangssatz:
    snippet = text
    
    prompt = f"""
    #Role
    You are a literary scholar
    
    #Instruction
    Your task is to recognize points of indeterminacy in German-language narrative texts given to you. The opening of a literary narrative has a point of indeterminacy precisely when it is characterized by a lack of information regarding the central character, the subject, the place or the time of the plot.
    Give first an analysis and then select one of the following German labels to classify the degree of the lack of information regarding the central character, the subject, the place or the time of the plot in the given text snippet:
        Label: sehr viel Information
        Label: viel Information
        Label: wenig Information
        Label: sehr wenig Information
    
    #Output
    Use the following output format:
    Analysis: str
    Label: str
    
    #
    Analyze the following German text: {snippet}
    Analysis: 
    Label:
    """
    
    result = get_completion(prompt)
    print(result)
    results.append(result)

Analysis: The given text snippet, "Am meisten war ich darüber erstaunt, wie schnell und einfach es ging," translates to "I was most astonished at how quickly and easily it went." This sentence does not provide any specific information about the central character, the subject, the place, or the time of the plot. The speaker's identity, the context of what went quickly and easily, the location, and the time frame are all indeterminate.

Label: sehr wenig Information
### Analysis:
The text snippet provides some specific information about the central character and the context. We know that the central character is named Renatus, and he is part of a military company that has just left the barracks yard. The subject of the narrative seems to be related to military life and duties, as indicated by Renatus needing to follow a regulation to put on a white drill uniform. The place is somewhat specified as a barracks yard and a room (referred to as "Stube"). However, the exact time of the plot is

In [12]:
results_1 = []
for text in results:
    if "stark unbestimmt" in text:
        results_1.append("stark unbestimmt")
    elif "wenig unbestimmt" in text:
        results_1.append("wenig unbestimmt")
    elif "nicht unbestimmt" in text:
        results_1.append("nicht unbestimmt")
    elif "unbestimmt" in text:
        results_1.append("unbestimmt")
    else:
        results_1.append("NaN")

In [13]:
results_1 = []
for text in results:
    if "sehr viel Information" in text:
        results_1.append("sehr viel Information")
    elif "viel Information" in text:
        results_1.append("viel Information")
    elif "sehr wenig Information" in text:
        results_1.append("sehr wenig Information")
    elif "wenig Information" in text:
        results_1.append("wenig Information")
    else:
        results_1.append("NaN")

In [14]:
results_se = pd.Series(results_1)

In [15]:
predictions = results_se
ground_truth = df.Klassifikation

In [16]:
pd.DataFrame(zip(ground_truth, predictions))

,0,1
0,sehr wenig Information,sehr wenig Information
1,wenig Information,wenig Information
2,wenig Information,sehr wenig Information
3,sehr wenig Information,sehr wenig Information
4,wenig Information,wenig Information
5,wenig Information,sehr wenig Information
6,wenig Information,sehr wenig Information
7,wenig Information,wenig Information
8,sehr wenig Information,sehr wenig Information
9,sehr wenig Information,sehr wenig Information


In [17]:
pd.DataFrame(list(zip([f1_score(ground_truth, predictions, average="weighted")],
                      [recall_score(ground_truth, predictions,  average="weighted")],
                      [precision_score(ground_truth, predictions, average="weighted")],
                      [accuracy_score(ground_truth, predictions,)])),
                      columns = ["F1", "Recall", "Precision", "Accuracy"])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,F1,Recall,Precision,Accuracy
0,0.445664,0.509091,0.419755,0.509091


In [18]:
print(classification_report(ground_truth, predictions))

                        precision    recall  f1-score   support

sehr wenig Information       0.54      0.84      0.66        25
      viel Information       0.00      0.00      0.00         8
     wenig Information       0.44      0.32      0.37        22

              accuracy                           0.51        55
             macro avg       0.33      0.39      0.34        55
          weighted avg       0.42      0.51      0.45        55



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.